In [68]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Carga de dataset

In [69]:
movies_ml = pd.read_csv(r'C:\Users\felip\Desktop\Stuff\Cursos\SoyHenry\Clases\LABS\PI ML Ops\pi_mlops_felipeviera\datasets\movies_ml.csv')

## EDA

In [70]:
movies_ml.columns

Index(['budget', 'id', 'popularity', 'release_date', 'revenue', 'runtime',
       'title', 'vote_average', 'vote_count', 'Action', 'Adventure',
       'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family',
       'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery',
       'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western',
       'African Languages', 'Asian Languages', 'Germanic Languages',
       'Other Languages', 'Romance Languages', 'Slavic Languages', 'Africa',
       'Antarctica', 'Asia', 'Europe', 'North America', 'Oceania',
       'Other Continent', 'South America', 'Canceled', 'In Progress',
       'Released', 'release_day', 'release_month', 'release_year', 'return'],
      dtype='object')

In [71]:
movies_ml = movies_ml[movies_ml['Canceled'] != 1]
movies_ml = movies_ml[movies_ml['In Progress'] != 1]
movies_ml = movies_ml[movies_ml['Antarctica'] != 1]
movies_ml = movies_ml[movies_ml['Other Continent'] != 1]

In [72]:
movies_ml.isna().sum()

budget                  0
id                      0
popularity              0
release_date            0
revenue                 0
runtime               244
title                   0
vote_average            0
vote_count              0
Action                  0
Adventure               0
Animation               0
Comedy                  0
Crime                   0
Documentary             0
Drama                   0
Family                  0
Fantasy                 0
Foreign                 0
History                 0
Horror                  0
Music                   0
Mystery                 0
Romance                 0
Science Fiction         0
TV Movie                0
Thriller                0
War                     0
Western                 0
African Languages       0
Asian Languages         0
Germanic Languages      0
Other Languages         0
Romance Languages       0
Slavic Languages        0
Africa                  0
Antarctica              0
Asia                    0
Europe      

In [73]:
movies_ml_details = movies_ml

In [74]:
movies_ml = movies_ml.drop(columns=['id', 'title', 'vote_average', 'vote_count', 'runtime', 'Other Continent', 'Antarctica', 'Canceled', 'In Progress',
       'Released', 'release_day', 'release_date', 'release_month'])

In [75]:
movies_ml_details.to_csv(r'C:\Users\felip\Desktop\Stuff\Cursos\SoyHenry\Clases\LABS\PI ML Ops\pi_mlops_felipeviera\datasets\movies_ml_details.csv')

In [76]:
movies_ml.to_csv(r'C:\Users\felip\Desktop\Stuff\Cursos\SoyHenry\Clases\LABS\PI ML Ops\pi_mlops_felipeviera\datasets\movies_ml_processed.csv')

## Modelo ML
### Preprocesamiento

In [77]:
from sklearn.preprocessing import MinMaxScaler

# Se normalizan datos númericos continuos para que todos contribuyan al modelo de manera equitativa
# Se inicializa el escalador
scaler = MinMaxScaler()

# Se seleccionan los datos númericos
columnas_numericas = ['budget', 'popularity', 'revenue', 'release_year', 'return']

# Se normaliza las columnas
# fit_transform calcula valores minimo y máximo de cada columna y las escala según corresponda
movies_ml[columnas_numericas] = scaler.fit_transform(movies_ml[columnas_numericas])

In [78]:
# Se crea una matriz de características
#   * La cual es un array 2D en el que cada fila representa una película 
#     y cada columna representa una característica.
# La similitud será calculada comparando filas de esta matriz

caracteristicas_columnas = [
    'budget', 'popularity', 'revenue', 'release_year', 'return',
    'Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
    'African Languages', 'Asian Languages',
    'Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America' 
]

# .values convierte este subset del dataframe en un Numpy array 
# que será usado en rápidas operaciones numéricas
matriz_caracteristicas = movies_ml[caracteristicas_columnas].values

In [79]:
# Se disminuye el peso de las columnas binarias

columnas_binarias = [
    'Action', 'Adventure', 'Animation', 'Comedy', 'Crime',
    'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign',
    'History', 'Horror', 'Music', 'Mystery', 'Romance',
    'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western',
    'African Languages', 'Asian Languages', 'Germanic Languages',
    'Other Languages', 'Romance Languages', 'Slavic Languages',
    'Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America'
]

# Se filtra columnas_binarias para incluir solo las que esten presentes en matriz_caracteristicas
columnas_presentes = [col for col in columnas_binarias if col in caracteristicas_columnas]

# Se obtienen indices de las columnas binarias
indices_binarias = [caracteristicas_columnas.index(col) for col in columnas_presentes]

# Se escalan las columnas binarias en matriz_caracteristicas
matriz_caracteristicas[:, indices_binarias] *= 0.3

In [80]:
from sklearn.decomposition import PCA

# Se reduce dimensionalidad con PCA

# Se reduce la dimensionalidad a 10 componentes
pca = PCA(n_components=10)
reduced_features = pca.fit_transform(matriz_caracteristicas)

matriz_caracteristicas = reduced_features

In [82]:
from sklearn.neighbors import NearestNeighbors

# Se hace fit a los vecinos más cercanos usando la similitud coseno
# Usando una busqueda "brute force" para mantener la simplicidad
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(matriz_caracteristicas)

NearestNeighbors(algorithm='brute', metric='cosine')

### Se crea función de recomendación

In [92]:
def recomendacion(titulo, n_neighbors=5):
    try:
        titulo = titulo.title()
        
        # Se busca el index de la película ingresada
        movie_index = movies_ml_details[movies_ml_details['title'] == titulo].index[0]

        # Se obtienen las distancias y los indices de las películas similares
        distancias, indices = model.kneighbors([matriz_caracteristicas[movie_index]], n_neighbors=n_neighbors+1)

        # Se excluye la pelicula ingresada por el usuario (1er resultado es siempre la película del usuario)
        similar_movies = indices[0][1:]
        similar_distances = distancias[0][1:]

        # Se obtienen los detalles de las películas similares
        recomendaciones = movies_ml_details.iloc[similar_movies][['title', 'popularity', 'release_year']].copy()
        recomendaciones['similarity_score'] = 1 - similar_distances # Convierte la distancia a puntaje de similitud

        return recomendaciones.reset_index(drop=True) # Se hace reset al index para un output más limpio
    except IndexError:
        return f"No se encontró una película con el título: {titulo}"
    except Exception as e:
        return f"Ha ocurrido un error: {e}"

In [89]:
recomendaciones = recomendacion("Alien")

print(recomendaciones)

                                 title  popularity  release_year  \
0           Ten Zan - Ultimate Mission    0.006091          1988   
1  The Legend Of The 7 Golden Vampires    1.845877          1974   
2              Don 2: The King Is Back    4.949640          2011   
3                         The Assassin    6.294815          2015   
4                         Amsterdamned    1.965789          1988   

   similarity_score  
0          0.998272  
1          0.989037  
2          0.944093  
3          0.927767  
4          0.895774  


In [91]:
recomendaciones = recomendacion("Toy Story")

print(recomendaciones)

                                title  popularity  release_year  \
0                    Cats Don'T Dance    3.406836          1997   
1                          Cool World   11.781895          1992   
2                    Oliver & Company   10.813579          1988   
3                            Garfield   10.003679          2004   
4  South Park: Bigger, Longer & Uncut   11.818055          1999   

   similarity_score  
0          0.994864  
1          0.993709  
2          0.989773  
3          0.988438  
4          0.987301  


In [90]:
recomendaciones = recomendacion("Inception")

print(recomendaciones)

                                            title  popularity  release_year  \
0                                 Miss Rose White    0.840651          1992   
1                                             Mac    0.810792          1992   
2                       Doing Time On Maple Drive    0.678176          1992   
3                                     Spellcaster    0.663414          1992   
4  Aileen Wuornos: The Selling Of A Serial Killer    1.138615          1992   

   similarity_score  
0               1.0  
1               1.0  
2               1.0  
3               1.0  
4               1.0  
